# LangChain Workshop

### What is LangChain?

LangChain is a library that provides a lot of helpful components for working with LLMs. These include:
* Model I/O
* Memory 
* Data connection 
* Chains
* Agents

### Model I/O

#### Use LangChains interface to make calls to LLM providers

In [ ]:
from langchain.llms import OpenAI

In [4]:
llm = OpenAI()
llm("Why did the chicken cross the road?")

'\n\nTo get to the other side.'

#### Use prompt templates to format input 

In [5]:
from langchain import PromptTemplate

In [20]:
template = """
Write a letter to {recipient} about {subject} from {sender}.

"""

prompt = PromptTemplate(
    input_variables=["recipient", "subject","sender"],
    template=template
)

letter_to_dad = prompt.format(recipient='Dad',subject='Las Vegas',sender='Max')

print(llm(letter_to_dad))



Dear Dad,

I had such a great time in Las Vegas! I'm so glad we decided to take the trip. There was so much to see and do that I couldn't believe it. I visited all the tourist attractions, like the Stratosphere and the Fremont Street Experience.

The shopping was great, too! I found some amazing bargains at the outlet malls. I even got some souvenirs to bring home. I'm sure you'll get a kick out of them.

I also had the chance to try my luck at the casinos. I didn't win big, but I had a blast playing the slots. I think I'm getting the hang of it.

I really appreciate you taking me to Las Vegas. I had so much fun and I learned a lot, too. I can't wait to come back and explore more of the city.

Love,

Max


#### Use Output Parsers to format LLM response into desired format

In [21]:
template = """
Given the following letter separated by backticks: 

```{letter}```

Extract the following information: 

sender: Who is this letter from?
location: What location is the letter about? 
recipient: Who is this letter addressed to?

Format the output as JSON with the following keys:
sender
location
recipient
"""
prompt = PromptTemplate.from_template(template)
message = prompt.format(letter=letter_to_dad)
response = llm(message)
print(response)


{"sender": "Max", "location": "Las Vegas", "recipient": "Dad"}


In [22]:
type(response)

str

In [27]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain import output_parser

ImportError: cannot import name 'output_parser' from 'langchain' (/Users/maxrivera/Desktop/langchain-demo/langchain-venv/lib/python3.9/site-packages/langchain/__init__.py)

In [25]:
sender_schema = ResponseSchema(name="sender",description="Who is this letter from?")
location_schema = ResponseSchema(name="location",description="What location is the letter about?")
recipien`t_schema = ResponseSchema(name="recipient",description="Who is this letter addressed to?")
response_schemas=[sender_schema, location_schema, recipient_schema]

In [30]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"sender": string  // Who is this letter from?
	"location": string  // What location is the letter about?
	"recipient": string  // Who is this letter addressed to?
}
```


In [31]:
template = """
Given the following letter separated by backticks: 

```{letter}```

Extract the following information: 

sender: Who is this letter from?
location: What location is the letter about? 
recipient: Who is this letter addressed to?

Format the output as JSON with the following keys:
sender
location
recipient

{format_instructions}
"""
prompt = PromptTemplate.from_template(template)
message = prompt.format(letter=letter_to_dad, format_instructions=format_instructions)
response = llm(message)
print(response)


```json
{
	"sender": "Max", 
	"location": "Las Vegas", 
	"recipient": "Dad"
}
```


In [33]:
output_dict = output_parser.parse(response)

In [37]:
output_dict

{'sender': 'Max', 'location': 'Las Vegas', 'recipient': 'Dad'}

In [36]:
type(output_dict)

dict

Load Documents

In [3]:
from langchain.document_loaders import TextLoader

In [25]:
loader = TextLoader("./birthday_ideas.txt")
doc = loader.load()
# print(doc)
# with open("./birthday_ideas.txt") as f:
#     birthday_doc = f.read()

Split text by tokens

In [45]:
from langchain.text_splitter import CharacterTextSplitter

In [61]:
text_splitter = CharacterTextSplitter(
chunk_size=5,
chunk_overlap=0,
length_function=len
)

In [62]:
texts = text_splitter.split_documents(doc)

In [63]:
texts

[Document(page_content="Birthdays in the family: \n- Zarah's birthday is in February \n- Max's birthday is in April \n- James' birthday is in May \n- Mom's birthday is in July \n- Dad's birthday is in October \n- Zyde's birthday is in December", metadata={'source': './birthday_ideas.txt'})]

Embed chunks 

In [22]:
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings

In [23]:
embeddings = HuggingFaceEmbeddings()

In [65]:
embedding_result = embeddings.embed_documents(texts)

AttributeError: 'Document' object has no attribute 'replace'